# 1.How big of an array can I sort using merge-sort within one minute?

Merge sort runs in O(n log n) time because at each “level” of recursion it does an O(n) merge, and there are log n levels from repeatedly splitting the array in half. 

A rough estimate of speed is that a program written in C or C++ would perform 10^8  high level operations such as addition, multiplication, comparison, reading memory, etc. in a second. Python, being an interpreted language, would be about 10 times slower (10^7).

Therefore, if I run this code on Python, in one minute I am able to do 10^7 * 60 high level operations, i.e 6 * 10^8 , while in C or C++ I can do 6* 10^9 high level operations.

So the equation to solve for Python is roughly n * log2(n)  = 6 * 10^8. This equation cannot be solved algebraically in a simple closed form, but we can approximate the solution numerically to n = 2.44 * 10 ^ 7 and so for C or C++ it is n = 2.44 * 10^8.

These are very rough ballpark figures and actual performance can vary significantly depending on hardware, compiler optimizations, memory speeds, and other implementation details.


## Python

In [ ]:
import random
import time
import sys

# --- MERGE SORT IMPLEMENTATION ---
def merge_sort(arr):
    """
    Recursively splits and merges the array.
    """
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        merge_sort(left_half)
        merge_sort(right_half)

        i = j = k = 0
        # Merge the two halves
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        # Copy the remaining elements of left_half, if any
        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        # Copy the remaining elements of right_half, if any
        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1

def main():
    """
    Usage: python merge_sort.py <n>
      - n is the size of the array to sort. Default = 1e6 (1 million).
    """
    if len(sys.argv) > 1:
        n = int(sys.argv[1])
    else:
        n = 10**6  # Default size if no argument is given

    print(f"Generating array of size {n} ...")
    arr = [random.randint(0, 10**9) for _ in range(n)]

    print("Starting merge sort...")
    start_time = time.time()
    merge_sort(arr)
    end_time = time.time()

    print(f"Time taken to sort {n} elements: {end_time - start_time:.4f} seconds")

    # Optional: check if array is sorted
    # (This can be expensive to do for large n, so you may remove this check.)
    assert all(arr[i] <= arr[i+1] for i in range(n-1)), "Array is not sorted!"

if __name__ == "__main__":
    main()


python 2-sem/CS/Homework/Merge_sort.py 100000

Generating array of size 100000 ...

Starting merge sort...

Time taken to sort 100000 elements: 0.3936 seconds

python 2-sem/CS/Homework/Merge_sort.py 1000000

Generating array of size 1000000 ...

Starting merge sort...

Time taken to sort 1000000 elements: 5.0034 seconds

### Let's try our theoretical limit 2.44 * 10^7

python 2-sem/CS/Homework/Merge_sort.py 244000000



It was taking quite a long time and to see that the code was actually running I have moved the terminal in a new window and I have run "top" in the new terminal to investigate my system status:

Processes: 381 total, 4 running, 377 sleeping, 3224 threads                                                    10:17:04

Load Avg: 2.87, 2.40, 2.47  CPU usage: 31.12% user, 7.72% sys, 61.15% idle --> My CPU is not maxed out. This confirms that the slowdown is due to memory swapping, not CPU constraints.


SharedLibs: 197M resident, 44M data, 13M linkedit. MemRegions: 877543 total, 1417M resident, 74M private, 
626M shared.

PhysMem: 7555M used (1338M wired, 3751M compressor), 75M unused.--> I only have 75MB of free RAM, which is not enough to handle a huge 244M-element array.
My system is relying on swap memory, which is extremely slow compared to RAM.

VM: 228T vsize, 4729M framework vsize, 2254854(258) swapins, 2953596(336) swapouts.--> Swap-ins and swap-outs indicate that my Mac is moving data between RAM and disk, which slows down my program significantly.

Networks: packets: 2902738/2780M in, 1361836/798M out. Disks: 14117688/269G read, 5768917/113G written.

Let's try to be me more conservative

python 2-sem/CS/Homework/Merge_sort.py 1500000

Generating array of size 1500000 ...

Starting merge sort...

Time taken to sort 1500000 elements: 11.7035 seconds

## Python on Google Colab

!python3 /content/Merge_sort.py 100000

Generating array of size 100000 ...

Starting merge sort...

Time taken to sort 100000 elements: 0.3144 seconds

!python3 /content/Merge_sort.py 1000000

Generating array of size 1000000 ...

Starting merge sort...

Time taken to sort 1000000 elements: 4.4232 seconds

### Let's try our theoretical limit 2.44 * 10^7

!python3 /content/Merge_sort.py 24400000

Generating array of size 24400000 ...

Starting merge sort...

Time taken to sort 24400000 elements: 173.0057 seconds

!python3 /content/Merge_sort.py 15000000

Generating array of size 15000000 ...

Starting merge sort...

Time taken to sort 15000000 elements: 100.2316 seconds

!python3 /content/Merge_sort.py 12500000

Generating array of size 12500000 ...

Starting merge sort...

Time taken to sort 12500000 elements: 83.0785 seconds

!python3 /content/Merge_sort.py 11000000

Generating array of size 11000000 ...

Starting merge sort...

Time taken to sort 11000000 elements: 73.2269 seconds

# This is very close!

!python3 /content/Merge_sort.py 10000000

Generating array of size 10000000 ...

Starting merge sort...

## Time taken to sort 10000000 elements: 65.3124 seconds

## C++

```cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <algorithm>  // For std::swap, if needed
#include <chrono>

using namespace std;

// --- MERGE SORT IMPLEMENTATION ---
void merge(vector<int> &arr, int left, int mid, int right) {
    int n1 = mid - left + 1;
    int n2 = right - mid;

    vector<int> L(n1), R(n2);

    for(int i = 0; i < n1; i++) {
        L[i] = arr[left + i];
    }
    for(int j = 0; j < n2; j++) {
        R[j] = arr[mid + 1 + j];
    }

    int i = 0, j = 0, k = left;
    while(i < n1 && j < n2) {
        if(L[i] <= R[j]) {
            arr[k++] = L[i++];
        } else {
            arr[k++] = R[j++];
        }
    }
    while(i < n1) {
        arr[k++] = L[i++];
    }
    while(j < n2) {
        arr[k++] = R[j++];
    }
}

void mergeSort(vector<int> &arr, int left, int right) {
    if(left < right) {
        int mid = left + (right - left) / 2;
        mergeSort(arr, left, mid);
        mergeSort(arr, mid + 1, right);
        merge(arr, left, mid, right);
    }
}

int main(int argc, char** argv) {
    // Default array size is 1e6 if no argument is provided
    long long n = 1000000;
    if(argc > 1) {
        n = atoll(argv[1]);
    }

    cout << "Generating array of size " << n << " ..." << endl;
    srand((unsigned) time(NULL));
    vector<int> arr(n);
    for(long long i = 0; i < n; i++) {
        arr[i] = rand(); // random int
    }

    cout << "Starting merge sort..." << endl;
    auto start = chrono::high_resolution_clock::now();
    mergeSort(arr, 0, n - 1);
    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double> diff = end - start;

    cout << "Time taken to sort " << n << " elements: " 
         << diff.count() << " seconds" << endl;

    return 0;
}


 g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort 

Generating array of size 1000000 ...

Starting merge sort...

Time taken to sort 1000000 elements: 0.141446 seconds

g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort 24400000

Generating array of size 24400000 ...

Starting merge sort...

Time taken to sort 24400000 elements: 4.26308 seconds

### Let's try our theoretical limit 2.44 * 10^8

g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort 244000000

Generating array of size 244000000 ...

Starting merge sort...

Time taken to sort 244000000 elements: 49.2603 seconds

Ok, we are not far

Let's push it a bit

g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort 300000000

Generating array of size 300000000 ...

Starting merge sort...

Time taken to sort 300000000 elements: 67.4735 seconds

Ok, we are very close let's try another one

g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort 280000000

Generating array of size 280000000 ...

Starting merge sort...

Time taken to sort 280000000 elements: 74.4835 seconds

g++ 2-sem/CS/Homework/merge_sort.cpp -o merge_sort -O2
./merge_sort

Generating array of size 280000000 ...

Starting merge sort...

Time taken to sort 280000000 elements: 86.0696 seconds

## Interpetation

The results show a **consistent but non-linear increase in sorting time** as we scale up the input size. L

Merge Sort runs in **\( O(n log n) \)** time complexity, meaning that sorting time should roughly follow:

\[
T(n) =  k * nlogn
\] (approximately)

where **\( k \)** is some constant dependent on hardware, compiler optimizations, and memory efficiency.

### Expected Time Ratios
If the sorting time follows \( O(n log n) \), then increasing \( n \) should increase time **slower than linearly**.

| **Input Size \( n \)** | **Expected Growth \( O(n \log n) \)** |
|----------------|------------------|
| \( 10^6 \) → \( 10^7 \) | **10x increase but ~14x time increase** |
| \( 10^7 \) → \( 10^8 \) | **10x increase but ~12x time increase** |
| \( 10^8 \) → \( 3 \times 10^8 \) | **3x increase but ~3.5x time increase** |

From your data, the **observed** increases in time are **reasonable** given the theoretical prediction.

### Results
| **Input Size** | **Time Taken (s)** | **Ratio Increase** |
|--------------|----------------|----------------|
| **1M**  | **0.14s**  | **Baseline** |
| **24.4M**  | **4.26s**  | \( \approx 30x \) |
| **244M**  | **49.26s**  | \( \approx 11.6x \) |
| **280M**  | **74.48s**  | \( \approx 1.5x \) |
| **300M**  | **86.07s**  | \( \approx 1.16x \) |

### Comparing 
- **Between `24.4M` and `244M`**:  
  - **Theoretical prediction**: Should take about **10-12x longer**  
  - **Observed time increase**: **11.6x** ✅ **Matches expectations!**
  
- **Between `244M` and `280M`**:  
  - **Expected:** Increase of **\( (280 \log 280) / (244 \log 244) \approx 1.4 \)**  
  - **Observed:** \( \approx 1.5x \) ✅ **Slightly higher, but still reasonable**  

- **Between `280M` and `300M`**:  
  - **Expected:** \( (300 \log 300) / (280 \log 280) \approx 1.15 \)  
  - **Observed:** \( \approx 1.16x \) ✅ **Very close!**

### Conclusion 

- **Merge Sort behaves as expected in \( O(n \log n) \)**.
- **Larger inputs grow sorting time sub-linearly**, meaning time doesn't scale exactly with \( n \), but slightly slower.
- **Our experiment confirms the theoretical limit is realistic**.

### Why Did Sorting `280M` Take Longer the Second Time?
Our **first run** of `280M` took **74.48 seconds**, but our **second run** took **86.07 seconds**.

**Possible reasons:**
1. **Memory Bottleneck**  
   - Large arrays take up **significant RAM**.
   - If our RAM was already partially filled, the second run may have used **swap memory (slower disk access)**.

2. **CPU Caching Effects**  
   - If **another background process** (e.g., macOS system tasks) was running, it might have affected performance.
   - **Modern CPUs optimize based on past runs**, but if cache gets flushed, the next run might be slower.

3. **Thermal Throttling**  
   - My **MacBook may have gotten hotter**, reducing CPU frequency.
   - Running `top` while sorting confirm this.

 

Remark: Expect that at some point, **RAM will become a bottleneck**. If **sorting time suddenly jumps drastically**, it's a sign that **macOS is swapping memory to disk** (which is very slow).

## C++ on Google Colab


!g++ /content/merge_sort.cpp -o /content/merge_sort -O2 -std=c++11


!/content/merge_sort 1000000

Generating array of size 1000000 ...
Starting merge sort...
Time taken to sort 1000000 elements: 0.225799 seconds

!g++ /content/merge_sort.cpp -o /content/merge_sort -O2 -std=c++11


!/content/merge_sort 24400000

Generating array of size 24400000 ...
Starting merge sort...
Time taken to sort 24400000 elements: 6.72649 seconds


!g++ /content/merge_sort.cpp -o /content/merge_sort -O2 -std=c++11


!/content/merge_sort 244000000

Generating array of size 244000000 ...
Starting merge sort...
Time taken to sort 244000000 elements: 70.6277 seconds

# This is very close!


!g++ /content/merge_sort.cpp -o /content/merge_sort -O2 -std=c++11


!/content/merge_sort 200000000



Generating array of size 200000000 ...

Starting merge sort...

## Time taken to sort 200000000 elements: 57.1181 seconds 


# 2. Gaussian elimination on an n-by-n matrix runs in time O(n^3). How long would it take to perform it on a matrix of size 2000x2000?


Gaussian elimination takes about O(n^3) time, so for a 2000 by 2000 matrix, that’s roughly 2000^3 = 8 × 10^9 operations. 

If Python handles about 10^7 operations per second, it would take around 800 seconds, which is over 13 minutes. 

In C or C++, which can do about 10^8 operations per second, it would take around 80 seconds. 

These are just rough estimates since actual speed depends on hardware and optimizations, but it gives a good idea of the difference.




## Python

Below is a very naive implementation of Gaussian elimination (without partial pivoting) to solve 
Ax=b, but here we’ll just focus on the matrix A. This will do roughly O(n^3) operations.

In [ ]:
import random
import time
import sys

def gaussian_elimination(A):
    """
    Perform naive Gaussian elimination in-place on matrix A of size n x n.
    This code does NOT do partial pivoting and does no checks for singular matrices.
    """
    n = len(A)
    for i in range(n):
        # Scale the pivot row (row i) so that the pivot element becomes 1
        pivot = A[i][i]
        for k in range(i, n):
            A[i][k] /= pivot

        # Eliminate below pivot
        for j in range(i+1, n):
            factor = A[j][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]

def main():
    """
    Usage: python gaussian_elimination.py <n>
      - n is the dimension of the matrix (n x n). Default = 2000.
    """
    if len(sys.argv) > 1:
        n = int(sys.argv[1])
    else:
        n = 2000

    # Generate a random n x n matrix (float entries)
    print(f"Generating a {n}x{n} matrix...")
    A = [[random.random() for _ in range(n)] for _ in range(n)]

    print("Starting Gaussian elimination...")
    start_time = time.time()
    gaussian_elimination(A)
    end_time = time.time()

    elapsed = end_time - start_time
    print(f"Time taken for {n}x{n} Gaussian elimination: {elapsed:.2f} seconds")

if __name__ == "__main__":
    main()


python 2-sem/CS/Homework/gaussian_elimination.py 2000

Processes: 458 total, 3 running, 455 sleeping, 3306 threads                                                    11:11
Load Avg: 2.57, 2.91, 3.21  CPU usage: 13.77% user, 1.43% sys, 84.79% idle
SharedLibs: 222M resident, 51M data, 17M linkedit. MemRegions: 876074 total, 1651M resident, 219M private, 835M shar
PhysMem: 7521M used (1414M wired, 3005M compressor), 107M unused.
VM: 257T vsize, 4729M framework vsize, 10119944(4) swapins, 10988175(0) swapouts.
Networks: packets: 2977552/2802M in, 1417936/815M out. Disks: 17137446/406G read, 8201056/239G written.

python 2-sem/CS/Homework/gaussian_elimination.py 2000

Generating a 2000x2000 matrix...

Starting Gaussian elimination...

Time taken for 2000x2000 Gaussian elimination: 363.32 seconds (6.05 minutes)

This is well below our theoretical prediction of 800 seconds (over 13 minutes)

## Python on Google Colab

!python3 /content/gaussian_elimination.py 2000

Generating a 2000x2000 matrix...

Starting Gaussian elimination...

Time taken for 2000x2000 Gaussian elimination: 268.07 seconds


# C++

```cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <chrono>
using namespace std;

int main(int argc, char** argv) {
    // Default size is 2000 if no argument is provided
    int n = 2000;
    if(argc > 1) {
        n = atoi(argv[1]);
    }

    cout << "Generating a " << n << "x" << n << " matrix of floats ..." << endl;
    srand((unsigned) time(NULL));

    vector<vector<double>> A(n, vector<double>(n));
    for(int i = 0; i < n; i++) {
        for(int j = 0; j < n; j++) {
            A[i][j] = static_cast<double>(rand()) / RAND_MAX; 
        }
    }

    cout << "Starting naive Gaussian elimination..." << endl;
    auto start = chrono::high_resolution_clock::now();

    // -- GAUSSIAN ELIMINATION (NAIVE, NO PIVOTING) --
    for(int i = 0; i < n; i++){
        double pivot = A[i][i];
        // Scale the pivot row
        for(int k = i; k < n; k++){
            A[i][k] /= pivot;
        }
        // Eliminate below pivot
        for(int j = i + 1; j < n; j++){
            double factor = A[j][i];
            for(int k = i; k < n; k++){
                A[j][k] -= factor * A[i][k];
            }
        }
    }

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double> diff = end - start;

    cout << "Time taken for " << n << "x" << n << " elimination: " 
         << diff.count() << " seconds" << endl;

    return 0;
}


g++ 2-sem/CS/Homework/gaussian_elimination.cpp -o gauss -O2
./gauss 2000

Generating a 2000x2000 matrix of floats ...
Starting naive Gaussian elimination...
Time taken for 2000x2000 elimination: 0.801322 seconds

That is 100 times faster than our prediction of 80 seconds

## C++ on Google Colab


!g++ /content/gaussian_elimination.cpp -o /content/gauss -O2 -std=c++11


!/content/gauss 2000

Generating a 2000x2000 matrix of floats ...
Starting naive Gaussian elimination...
Time taken for 2000x2000 elimination: 3.6029 seconds